<a href="https://colab.research.google.com/github/MattWroclaw/neural-networks/blob/main/05_regression/01_housing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem regresji - prognoza ceny mieszkań

Celem tego notebook'a jest zapoznanie się z problemami regresji. W odróżnieniu od problemów klasyfikacji, gdzie staramy się przyporządkować daną klasę, w modelach regresji przewidujemy ciągłą zmienną docelową.

### Spis treści:
1. [Import bibliotek](#a0)
2. [Załadowanie danych i wstępna eksploracja](#a1)
3. [Podział na zbiór treningowy oraz testowy](#a2)
4. [Standaryzacja danych](#a3)
5. [Budowa modelu](#a4)
6. [Trenowanie sieci](#a5)





In [56]:

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.17.0'

### <a name='a1'></a> 2. Załadowanie danych i wstępna eksploracja

In [57]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [58]:
dataset = raw_dataset.copy()
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [59]:
dataset.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


In [60]:
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.010029
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [61]:
dataset.dropna(inplace=True)

dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.000000
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [62]:
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [63]:
dataset.describe(include=['object'])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [64]:
dataset.ocean_proximity.value_counts()

,count
ocean_proximity,
<1H OCEAN,9034
INLAND,6496
NEAR OCEAN,2628
NEAR BAY,2270
ISLAND,5


In [65]:
px.histogram(dataset, x='median_house_value')

In [66]:
dataset.median_house_value.value_counts()

,count
median_house_value,
500001.000000,958
137500.000000,119
162500.000000,116
112500.000000,103
187500.000000,92
...,...
359200.000000,1
51200.000000,1
39800.000000,1


In [67]:
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

In [68]:
# Teraz zajmiemy się typem Object: ISLAND, NEAR_BAY, NEAR_OCEAN

dataset_dummies = pd.get_dummies(dataset, drop_first=True).astype(int)
dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122,37,41,880,129,322,126,8,452600,0,0,1,0
1,-122,37,21,7099,1106,2401,1138,8,358500,0,0,1,0
2,-122,37,52,1467,190,496,177,7,352100,0,0,1,0
3,-122,37,52,1274,235,558,219,5,341300,0,0,1,0
4,-122,37,52,1627,280,565,259,3,342200,0,0,1,0


The line dataset_dummies = pd.get_dummies(dataset, drop_first=True) performs one-hot encoding on the categorical features of the dataset and drops the first category for each feature.

Here's a breakdown:

    pd.get_dummies(): This function is part of the Pandas library and is used for one-hot encoding. One-hot encoding converts categorical features into numerical representations by creating new columns for each unique category.

    dataset: This is the input DataFrame containing the data with categorical features.

    drop_first=True: This argument drops the first category for each feature during one-hot encoding. This is done to avoid multicollinearity, which can cause issues in some machine learning models.


### <a name='a2'></a> 3. Podział na zbiór treningowy oraz testowy

In [69]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


Okay, let's break down the line of code:

train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)

Use code with caution

This line of code is creating a training dataset by randomly sampling 80% of the data from the dataset_dummies DataFrame.

Here's a breakdown of the components:

    train_dataset: This is the variable that will store the new training dataset.
    dataset_dummies.sample(): This is the function call that does the sampling.
        frac=0.8: This argument specifies that 80% of the data should be included in the sample.
        random_state=0: This argument sets a seed for the random number generator. This ensures that the same random sample will be generated ever

Okay, let's analyze the code:

test_dataset = dataset_dummies.drop(train_dataset.index)

Use code with caution

This line creates the test dataset by removing the rows used in the training dataset (train_dataset) from the original dataset (dataset_dummies).

Here's a breakdown:

    test_dataset: This is the variable that will store the new test dataset.
    dataset_dummies.drop(): This function is used to remove rows or columns from a Pandas DataFrame. In this case, it's removing rows.
    train_dataset.index: This part provides the indices (row labels) of the data points that were selected for the training dataset.


In [70]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122,38,18,3364,501,1442,506,6,313000,0,0,0,0
1859,-124,41,21,2696,578,1208,494,2,122400,0,0,0,1
51,-122,37,43,1868,456,1061,407,1,93800,0,0,1,0
11192,-117,33,28,2444,555,1848,567,3,198800,0,0,0,0
20355,-118,34,16,1807,346,587,296,1,162500,0,0,0,0


In [71]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122,37,52,2202,434,910,402,3,281500,0,0,1,0
13,-122,37,52,696,191,345,174,2,191300,0,0,1,0
19,-122,37,52,1503,298,690,275,2,162900,0,0,1,0
26,-122,37,49,1130,244,607,239,2,93800,0,0,1,0
40,-122,37,52,1665,419,946,395,2,155400,0,0,1,0


In [72]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [73]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000000,-119.132413,2.006918,-124.000000,-121.000000,-118.000000,-117.000000,-114.000000
latitude,15580.000000,35.118678,2.157684,32.000000,33.000000,34.000000,37.000000,41.000000
housing_median_age,15580.000000,28.489217,12.505895,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,15580.000000,2620.014506,2195.654212,2.000000,1441.000000,2112.000000,3119.000000,39320.000000
total_bedrooms,15580.000000,539.900578,424.499548,2.000000,299.000000,436.000000,647.000000,6445.000000
population,15580.000000,1441.193068,1160.542775,3.000000,801.000000,1179.000000,1746.000000,35682.000000
households,15580.000000,501.099487,385.039835,2.000000,283.000000,411.000000,605.000000,6082.000000
median_income,15580.000000,3.194288,1.603700,0.000000,2.000000,3.000000,4.000000,15.000000
ocean_proximity_INLAND,15580.000000,0.333569,0.471503,0.000000,0.000000,0.000000,1.000000,1.000000
ocean_proximity_ISLAND,15580.000000,0.000193,0.013876,0.000000,0.000000,0.000000,0.000000,1.000000


In [74]:
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

Ten fragment kodu jest częścią przygotowania danych do trenowania modelu uczenia maszynowego. W szczególności wykonuje on następujące operacje:

### Wyjaśnienie krok po kroku:

1. **`train_dataset.pop('median_house_value')` oraz `test_dataset.pop('median_house_value')`:**
   - W tej linii z zestawu danych (zarówno `train_dataset`, jak i `test_dataset`) usuwana jest kolumna o nazwie **`median_house_value`**.
   - Metoda `.pop()` usuwa kolumnę z ramki danych (DataFrame) i zwraca jej wartości, co oznacza, że po tej operacji dane w tej kolumnie zostaną przypisane do zmiennych `train_labels` oraz `test_labels`.

2. **Dlaczego usuwamy kolumnę?**
   - Kolumna **`median_house_value`** zawiera wartości docelowe, czyli etykiety, które chcemy przewidywać za pomocą modelu. W kontekście problemu przewidywania cen mieszkań, wartości te reprezentują medianę cen domów.
   - Ponieważ ta kolumna stanowi etykiety (target), jest usuwana z zestawu cech (danych wejściowych do modelu), a jej wartości są przechowywane osobno w zmiennych `train_labels` oraz `test_labels`, które będą wykorzystywane jako prawdziwe wartości do nauki modelu oraz do testowania jego skuteczności.

3. **Podział na zbiór treningowy i testowy:**
   - Dane zostały podzielone na zestaw treningowy (`train_dataset`) oraz testowy (`test_dataset`), co jest standardową praktyką przy uczeniu modeli. Zestaw treningowy służy do trenowania modelu, a zestaw testowy do oceny jego działania na nieznanych danych.

### Ostatecznie:

- `train_labels` i `test_labels` będą zawierać wartości median cen mieszkań, które są wartościami docelowymi, a `train_dataset` i `test_dataset` będą zawierać pozostałe kolumny, które będą używane jako cechy do nauki modelu.

### <a name='a3'></a> 4. Standaryzacja danych

In [75]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

Metoda `norm(x)` normalizuje dane, czyli przekształca je w taki sposób, aby miały średnią równą 0 i odchylenie standardowe równe 1. Normalizacja jest często stosowana w uczeniu maszynowym, aby dane wejściowe były porównywalne, co może pomóc w szybszym i bardziej efektywnym trenowaniu modelu.

### Wyjaśnienie krok po kroku:

1. **Argument `x`:**
   - To wejściowa wartość lub zbiór wartości (np. kolumna danych), które chcesz znormalizować.

2. **`train_stats['mean']` i `train_stats['std']`:**
   - Zakładamy, że wcześniej został obliczony słownik statystyk dla zbioru treningowego (`train_stats`).
   - `train_stats['mean']` zawiera średnią wartość dla każdej kolumny w zbiorze treningowym.
   - `train_stats['std']` zawiera odchylenie standardowe dla każdej kolumny w zbiorze treningowym.

3. **Obliczenie normalizacji:**
   - Normalizacja danych jest wykonywana według wzoru:
     \[
     \text{norm}(x) = \frac{x - \text{mean}}{\text{std}}
     \]
   - W tej metodzie, dla każdej wartości `x`, odejmowana jest średnia wartość danej cechy (kolumny) (`train_stats['mean']`), a następnie wynik dzielony przez odchylenie standardowe tej samej cechy (`train_stats['std']`).
   - Dzięki temu uzyskujemy znormalizowaną wartość, która ma średnią 0 i odchylenie standardowe 1.

### Dlaczego to jest ważne?
- **Skalowanie cech:** Modele uczenia maszynowego (np. sieci neuronowe, regresja liniowa) działają lepiej, gdy dane są znormalizowane. Dzięki temu model nie jest zdominowany przez cechy o większych wartościach (np. liczby mogą się różnić skalą: np. powierzchnia w m² i liczba pokoi).
- **Szybsza i bardziej stabilna konwergencja:** Normalizacja sprawia, że model może szybciej znaleźć optymalne rozwiązanie, ponieważ cechy są porównywalne w skali.

### Przykład:
Jeśli jedna z cech to powierzchnia domu, a średnia powierzchnia w zbiorze treningowym wynosi 150 m² z odchyleniem standardowym 50 m², to dla domu o powierzchni 200 m² normalizacja będzie wyglądać tak:
\[
\text{norm}(200) = \frac{200 - 150}{50} = 1
\]
Znormalizowana wartość wynosi 1.

In [76]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [77]:
normed_train_data.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


In [78]:
normed_train_data.head(3)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-1.428851,1.335378,-0.838742,0.338845,-0.091639,0.000695,0.012727,1.749525,-0.707459,-0.013877,-0.346133,-0.378230
1859,-2.425404,2.725758,-0.598855,0.034607,0.089751,-0.200934,-0.018438,-0.744708,-0.707459,-0.013877,-0.346133,2.643727
51,-1.428851,0.871918,1.160315,-0.342501,-0.197646,-0.327599,-0.244389,-1.368266,-0.707459,-0.013877,2.888874,-0.378230


In [79]:
normed_test_data.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.428851,0.871918,1.879976,-0.190383,-0.249472,-0.457711,-0.257375,-0.121150,-0.707459,-0.013877,2.888874,-0.378230
13,-1.428851,0.871918,1.879976,-0.876283,-0.821910,-0.944552,-0.849521,-0.744708,-0.707459,-0.013877,2.888874,-0.378230
19,-1.428851,0.871918,1.879976,-0.508739,-0.569849,-0.647277,-0.587211,-0.744708,-0.707459,-0.013877,2.888874,-0.378230


In [80]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values
print(normed_test_data)
print(normed_train_data)

[[-1.428851116375  0.871917581847  1.879976070113 ... -0.013877293686  2.888874361871 -0.378229535826]
 [-1.428851116375  0.871917581847  1.879976070113 ... -0.013877293686  2.888874361871 -0.378229535826]
 [-1.428851116375  0.871917581847  1.879976070113 ... -0.013877293686  2.888874361871 -0.378229535826]
 ...
 [-0.930574589457  1.798837572647 -0.678817233469 ... -0.013877293686 -0.346133368881 -0.378229535826]
 [-0.930574589457  1.798837572647 -0.998666396417 ... -0.013877293686 -0.346133368881 -0.378229535826]
 [-0.930574589457  1.798837572647 -0.279005779784 ... -0.013877293686 -0.346133368881 -0.378229535826]]
[[-1.428851116375  1.335377577247 -0.838741814943 ... -0.013877293686 -0.346133368881 -0.378229535826]
 [-2.425404170211  2.725757563448 -0.598854942732 ... -0.013877293686 -0.346133368881  2.64372747349 ]
 [-1.428851116375  0.871917581847  1.16031545348  ... -0.013877293686  2.888874361871 -0.378229535826]
 ...
 [ 0.564254991298 -0.981922399754 -0.279005779784 ... -0.01387

### <a name='a4'></a> 5. Budowa modelu

In [81]:
def build_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

In [82]:
model = build_model()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │          13,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

### <a name='a5'></a> 6. Trenowanie sieci

In [83]:
history = model.fit(
    normed_train_data,
    train_labels.values,
    # epochs=150,
    epochs = 20,
    validation_split=0.2,
    verbose=1, batch_size=32)

Epoch 1/20
390/390 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 31056889856.0000 - mae: 143150.3125 - mse: 31056889856.0000 - val_loss: 4872975360.0000 - val_mae: 50581.4102 - val_mse: 4872975360.0000
Epoch 2/20
390/390 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 4590851072.0000 - mae: 49055.9336 - mse: 4590851072.0000 - val_loss: 4032856832.0000 - val_mae: 47224.0469 - val_mse: 4032856832.0000
Epoch 3/20
390/390 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 4042124800.0000 - mae: 45990.5430 - mse: 4042124800.0000 - val_loss: 3832527360.0000 - val_mae: 45709.1367 - val_mse: 3832527360.0000
Epoch 4/20
390/390 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - loss: 3919965696.0000 - mae: 45327.0859 - mse: 3919965696.0000 - val_loss: 3752738048.0000 - val_mae: 45469.3164 - val_mse: 3752738048.0000
Epoch 5/20
390/390 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 3771826688.0000 - mae: 44599.5898 - mse: 3771826688.0000 - val_loss: 3766384384.0000 - val_mae: 45593.4727 - val_mse: 3766384384.0000
Epoch 6/20
390/390 ━━

In [84]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [85]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3270459904.0000 - mae: 41958.6328 - mse: 3270459904.0000
loss    3406912000.0000
compile_metrics42444.1484


In [86]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([250454.67 , 195982.55 , 195793.55 , ...,  97583.1  , 126955.055,  75452.02 ], dtype=float32)

In [87]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500,250454.671875
13,191300,195982.546875
19,162900,195793.546875
26,93800,183636.437500
40,155400,201267.187500


In [88]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [89]:
pred.head()

,median_house_value,predictions
10,281500,250454.671875
13,191300,195982.546875
19,162900,195793.546875
26,93800,183636.437500
40,155400,201267.187500


In [90]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500,250454.671875,31045.328125
13,191300,195982.546875,-4682.546875
19,162900,195793.546875,-32893.546875
26,93800,183636.437500,-89836.437500
40,155400,201267.187500,-45867.187500


In [91]:
px.histogram(pred, 'error', marginal='rug', width=1000)